In [1]:
from bs4 import BeautifulSoup # biblioteca para análisis y extracción de datos de archivos HTML y XML.
import requests
import pandas as pd
import numpy
import re

In [2]:
url = 'https://www.cowi.com/about/news-and-press/comparing-co2-emissions-from-different-energy-sources'
response = requests.get(url)
print(response.status_code)

200


In [3]:
# html_doc -> res.content
soup = BeautifulSoup(response.content, 'html.parser')
print(soup)


<!DOCTYPE html>

<html class="no-js" lang="en">
<head>
<!-- PD0MDWK00003Y -->
<script data-blockingmode="auto" data-cbid="0f8a53ec-9cd5-47f9-8c3c-2b286c927fb4" id="Cookiebot" src="https://consent.cookiebot.com/uc.js" type="text/javascript"></script>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="/favicon.ico?v=2" rel="shortcut icon" type="image/x-icon"/>
<title>Comparing CO2 emissions from different energy sources</title>
<meta content="When you produce electricity, you generate greenhouse gas emissions – and this is true for fossil-free energy sources as well. Even if the actual production of electricity does not generate emissions, there is a life-cycle perspective to consider." name="description"/>
<meta content="INDEX, FOLLOW" name="robots">
<meta content="Comparing CO2 emissions from different energy sources" property="og:title">
<meta content="COWI" property="og:site

In [5]:
#co2_data = soup.findAll('div', attrs={'class': 'small-12 medium-centered medium-8'})[2]
#co2_data


In [42]:
co2_data = soup.find_all("ul")[7]

In [43]:
list_items = co2_data.find_all('li')

# Initialize an empty list to store tuples
data_list = []

# Iterate over the list items and extract the power type and CO2 data
for item in list_items:
    # Extract text from the list item
    text = item.get_text()
    # Split the text at the colon to separate power type and CO2 data
    power_type, co2_data = text.split(':')
    # Append the extracted data as a tuple
    data_list.append((power_type.strip(), co2_data.strip()))
data_list

[('Hydropower', 'approximately 4 g CO2e/kWh'),
 ('Wind power', 'approximately 11 g CO2e/kWh'),
 ('Nuclear power', 'approximately 12 g CO2e/kWh'),
 ('Solar power', 'around 41 g CO2e/kWh'),
 ('Natural gas', '290-930 g CO2e/kWh'),
 ('Oil', '510-1170 g CO2e/kWh'),
 ('Coal', '740-1689 g CO2e/kWh.')]

In [46]:
co2_dict = {"Type of Power": [], "CO2e/kWh": []}

# Iterate over the data list and extract power type and CO2 value
for power, co2 in data_list:
    co2_value = re.findall(r'\b\d+(?:-\d+)?\b', co2)
    # Join numerical values if there are multiple
    co2_value = '-'.join(co2_value) if len(co2_value) > 1 else co2_value[0]
    co2_dict["Type of Power"].append(power)
    co2_dict["CO2e/kWh"].append(co2_value)
df_co2_data = pd.DataFrame(co2_dict)

# Display the DataFrame
df_co2_data

,Type of Power,CO2e/kWh
0,Hydropower,4
1,Wind power,11
2,Nuclear power,12
3,Solar power,41
4,Natural gas,290-930
5,Oil,510-1170
6,Coal,740-1689


In [47]:
df_co2_data.to_csv('COWIco2Data.csv', index=False)